# Funciones de Agregación

**SQL** tiene funciones que nos ayudan a realizar algunos cálculos relacionado con _promedios_, _mínimos_, _máximos_, _etc_,_etc_...

### `AVG` (_promedio_)

El promedio se calcula de la siguiente manera;
```sql
SELECT AVG(budget)
FROM films;
```
—_`AVG` nos arroja la media (promedio) de los presupuestos para todas las películas de la tabla `films`._ —

### `MAX` 

```sql
SELECT MAX(budget)
FROM films;
```
—_La consulta anterior tenía por objetivo rrojarnos el máximo valor para el atribudo `budget` en la tabla `films`._ —

Lo mismo pasaría con `MIN`.

—_Calculemos la suma de la duración de las películas de la tabla films:_ — 

In [26]:
import pandas as pd
import psycopg2.extras
conn = psycopg2.connect("dbname='test' user='test' host='/tmp/'")
cur = conn.cursor()

In [6]:
cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
cur.execute(
    """
    SELECT SUM (films.duration) FROM intro_sql.films;
    """
)
data = cur.fetchall()
conn.commit()
cur.close()
pd.DataFrame([i.copy() for i in data])

,sum
0,534882


### aggregate functions y `WHERE`

Podemos obviamente combinar estas dos _keywords_ para poder realizar filtros. Por ejemplo:

—_Obtener el total de presupuestos para todas las películas estrenadas después del 2010:_ — 
```sql
SELECT SUM(budget)
FROM films
WHERE release_year >= 2010;
```
—_El monto promedio recaudado para de las películas que se titulan iniciando con `A`;_ — 

In [9]:
cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
cur.execute(
    """
    SELECT AVG(films.gross)
    FROM intro_sql.films
    WHERE title LIKE 'A%';
    """
)
data = cur.fetchall()
conn.commit()
cur.close()
pd.DataFrame([i.copy() for i in data])

,avg
0,47893236.422480620155


—_La peor cantidad recaudada para las películas estrenadas en 1994:_ — 

In [11]:
cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
cur.execute(
        """
    SELECT MIN(films.gross)
    FROM intro_sql.films
    WHERE release_year=1994;
    """
)    
data = cur.fetchall()
conn.commit()
cur.close()
pd.DataFrame([i.copy() for i in data])

,min
0,125169


—_La mejor cantidad recaudada para las películas estrenadas entre 2000 y 2012:_ — 

In [13]:
cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
cur.execute(
        """
    SELECT MAX(films.gross)
    FROM intro_sql.films
    WHERE release_year BETWEEN 2000 AND 2012;
    """
)    
data = cur.fetchall()
conn.commit()
cur.close()
pd.DataFrame([i.copy() for i in data])

,max
0,760505847


Como podremos observar las _aggregate functions_ **no son permitidas** dentro de la clausula `WHERE`:

In [20]:
cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
cur.execute(
        """
    SELECT films.title
    FROM intro_sql.films
    WHERE MIN(gross);
    """
)    
data = cur.fetchall()
conn.commit()
cur.close()
pd.DataFrame([i.copy() for i in data])

GroupingError: aggregate functions are not allowed in WHERE
LINE 4:     WHERE MIN(gross);
                  ^


# Alias

Podemos renombrar las consultas realizadas mediante el _key word_ `AS`. Imaginemos que realizamos la siguiente consulta:

```sql
SELECT MAX(budget) AS max_budget,
       MAX(duration) AS max_duration
FROM films;
```
Podemos observar que las consultas estan siendo renombradas despues de `AS`. Lo que quiere decir que el resultado se mostrará como `max_budget` o `max_duration`. 

El resultado se muestra a continuación:

In [23]:
cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
cur.execute(
        """
    SELECT MAX(films.budget) AS max_budget,
    MAX(films.duration) AS max_duration
    FROM intro_sql.films;
    """
)    
data = cur.fetchall()
conn.commit()
cur.close()
pd.DataFrame([i.copy() for i in data])

,max_budget,max_duration
0,12215500000,334


—_La ganancia total de una película se obtiene al restar el monto recaudado (`gross`) menos el presupuesto `(budget)`. La siguiente consulta obtiene el nombre de la película y su ganancia total, renombrada como `net_profit`_ — 

In [24]:
cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
cur.execute(
    """
    SELECT films.title, (films.gross-films.budget) AS net_profit
    FROM intro_sql.films;
    """
)    
data = cur.fetchall()
conn.commit()
cur.close()
pd.DataFrame([i.copy() for i in data])

,title,net_profit
0,Intolerance: Love's Struggle Throughout the Ages,NaN
1,Over the Hill to the Poorhouse,2900000.0
2,The Big Parade,NaN
3,Metropolis,-5973565.0
4,Pandora's Box,NaN
5,The Broadway Melody,2429000.0
6,Hell's Angels,NaN
7,A Farewell to Arms,NaN
8,42nd Street,1861000.0
9,She Done Him Wrong,NaN


—_El promedio de duración de las películas mostrado como `avg_duration_hours` es igual a:_ — 

In [27]:
cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
cur.execute(
    """
    SELECT AVG(films.duration) /60.0  AS avg_duration_hours FROM intro_sql.films;
    """
)    
data = cur.fetchall()
conn.commit()
cur.close()
pd.DataFrame([i.copy() for i in data])

,avg_duration_hours
0,1.7991321897073663


Aquí haremos un paréntesis.

Las operaciones básicas como _suma_, _resta_, _multiplicación_ y _divición_ son posible de realizar en _PostgreSQL_. Pero hay que tener algo muy en cuenta.

—_Postgres asume que cuando dividimos lo realizamos con cantidades enteras siempre. De tal modo que será necesario siempre que realicemos una operación de estas que, al menos una de nuestras cantidades tenga una posición extra como decimal. Por ejemplo:_ — 

<img src="error_division.png"/>

La elección de la cifra que llevará la posición extra del decimal no importa.

—_Encontraremos el porcentaje de las personas que ya no están vivas de la tabla `people`:_ — 

In [28]:
cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
cur.execute(
    """
    SELECT COUNT(people.deathdate) * 100.0 / COUNT(*) AS percentage_dead FROM intro_sql.people;
    """
)    
data = cur.fetchall()
conn.commit()
cur.close()
pd.DataFrame([i.copy() for i in data])

,percentage_dead
0,9.3723949029415267


In [ ]:
cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
cur.execute(
    """
    SELECT COUNT(people.deathdate) * 100.0 / COUNT(*) AS percentage_dead FROM intro_sql.people;
    """
)    
data = cur.fetchall()
conn.commit()
cur.close()
pd.DataFrame([i.copy() for i in data])

—_Si quisieramos ver cual es el número equivalente en décadas que cubre la colección filmográfica de la tabla `films`:_ — 

In [29]:
cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
cur.execute(
    """
    SELECT (MAX(films.release_year)-MIN(films.release_year))/10.0 AS number_of_decades FROM intro_sql.films;
    """
)    
data = cur.fetchall()
conn.commit()
cur.close()
pd.DataFrame([i.copy() for i in data])

,number_of_decades
0,10.0000000000000000
